# Notebook: Evaluate Geocentric Model

Loads the model from `../models` and evaluates it.

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import plotly.graph_objects as go

# Import local library from Parent Dir
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import stars_utils

print("Libraries Loaded.")

In [ ]:
# Load Artifacts
model_dir = '../models'
model = tf.keras.models.load_model(os.path.join(model_dir, 'mars_geocentric_v2.keras'))
scaler = stars_utils.load_scaler(os.path.join(model_dir, 'scaler_features.pkl'))
y_scaler = stars_utils.load_scaler(os.path.join(model_dir, 'scaler_targets.pkl'))

# Load Data
df = pd.read_csv('../mars_processed_data.csv')

FEATURES = [
    'Time_Index', 'Time_Index_2', 
    'Sin_Mars', 'Cos_Mars',
    'Sin_Jupiter', 'Cos_Jupiter', 'Inv_Dist_Jupiter',
    'Sin_Saturn', 'Cos_Saturn', 'Inv_Dist_Saturn',
    'Sin_Venus', 'Cos_Venus', 'Inv_Dist_Venus',
    'X_au_Lag1', 'Y_au_Lag1', 'Z_au_Lag1', 
    'X_au_Lag2', 'Y_au_Lag2', 'Z_au_Lag2',
    'Kepler_X', 'Kepler_Y', 'Kepler_Z'     
]

split_index = int(len(df) * 0.8)
test_df = df.iloc[split_index:].copy()

X_test_scaled = scaler.transform(test_df[FEATURES].values)
pred_res = y_scaler.inverse_transform(model.predict(X_test_scaled, verbose=0))

test_df['Pred_X'] = test_df['Kepler_X'] + pred_res[:, 0]
test_df['Pred_Y'] = test_df['Kepler_Y'] + pred_res[:, 1]
test_df['Pred_Z'] = test_df['Kepler_Z'] + pred_res[:, 2]

mae = np.mean(np.sqrt((test_df['Pred_X']-test_df['X_au'])**2 + 
                      (test_df['Pred_Y']-test_df['Y_au'])**2 + 
                      (test_df['Pred_Z']-test_df['Z_au'])**2))
print(f"MAE: {mae:.6f} AU")

In [ ]:
fig = go.Figure()
viz_df = test_df.sample(2000).sort_index()
fig.add_trace(go.Scatter3d(x=viz_df['X_au'], y=viz_df['Y_au'], z=viz_df['Z_au'], mode='lines', line=dict(color='gray'), name='True'))
fig.add_trace(go.Scatter3d(x=viz_df['Pred_X'], y=viz_df['Pred_Y'], z=viz_df['Pred_Z'], mode='markers', marker=dict(size=3,color='lime'), name='AI'))
fig.update_layout(title=f'Test Set Orbit (MAE: {mae:.6f})', template='plotly_dark')
fig.write_html('../geocentric_viz_final.html')
print("Saved ../geocentric_viz_final.html")